In [27]:
import pandas as pd

# ============================= Cadastro e login =============================

usuarios = [{
    "nome": "admin",
    "senha": "admin",
    "tempo": 24,  # horas por dia
    "nivel": "básico"
}]

def cadastrar_usuario():
    nome = input("Digite seu nome: ")
    senha = input("Crie uma senha: ")
    tempo = int(input("Quanto tempo (em horas) você tem disponível por dia para estudar? "))
    nivel = input("Qual seu nível de conhecimento? (básico/intermediário/avançado): ")
    usuarios.append({"nome": nome, "senha": senha, "tempo": tempo, "nivel": nivel})
    print("✅ Cadastro realizado com sucesso!\n")

def login():
    print("=== Login ===")
    print("""
Usuário padrão:
  nome: admin
  senha: admin
""")
    nome = input("Nome de usuário: ")
    senha = input("Senha: ")
    for user in usuarios:
        if user["nome"] == nome and user["senha"] == senha:
            print(f"✅ Login bem-sucedido! Bem-vindo(a), {nome}\n")
            return user
    print("❌ Usuário ou senha incorretos.\n")
    return None


# ============================= Dados dos cursos =============================

cursos = [
    {"nome": "Python", "impacto": 9, "custo": 15},
    {"nome": "IA Ética", "impacto": 7, "custo": 4},
    {"nome": "Excel Avançado", "impacto": 5, "custo": 5},
    {"nome": "Power BI", "impacto": 8, "custo": 18},
    {"nome": "Machine Learning", "impacto": 10, "custo": 30},
    {"nome": "SQL", "impacto": 6, "custo": 25},
    {"nome": "Gestão Ágil", "impacto": 5, "custo": 10},
    {"nome": "Design Thinking", "impacto": 7, "custo": 5},
    {"nome": "Comunicação Profissional", "impacto": 6, "custo": 2},
    {"nome": "Inglês Técnico", "impacto": 8, "custo": 4},
    {"nome": "Finanças para TI", "impacto": 5, "custo": 8},
    {"nome": "Docker Básico", "impacto": 7, "custo": 3},
    {"nome": "Kubernetes", "impacto": 9, "custo": 5},
    {"nome": "Java Avançado", "impacto": 8, "custo": 20},
    {"nome": "Git e GitHub", "impacto": 6, "custo": 5},
    {"nome": "Redes de Computadores", "impacto": 7, "custo": 10},
    {"nome": "Segurança da Informação", "impacto": 8, "custo": 30},
    {"nome": "Liderança Técnica", "impacto": 7, "custo": 3},
    {"nome": "APIs com FastAPI", "impacto": 9, "custo": 4},
    {"nome": "Data Science", "impacto": 10, "custo": 5}
]

df_cursos = pd.DataFrame(cursos)


# ============================= Merge Sort =============================

def merge_sort(df, coluna, decrescente=True):
    lista = df.to_dict('records')
    if len(lista) <= 1:
        return pd.DataFrame(lista)
    meio = len(lista) // 2
    esquerda = merge_sort(pd.DataFrame(lista[:meio]), coluna, decrescente)
    direita = merge_sort(pd.DataFrame(lista[meio:]), coluna, decrescente)
    return merge(esquerda, direita, coluna, decrescente)

def merge(esq, dir, coluna, decrescente):
    e = esq.to_dict('records')
    d = dir.to_dict('records')
    res = []
    i = j = 0
    while i < len(e) and j < len(d):
        cond = (e[i][coluna] > d[j][coluna]) if decrescente else (e[i][coluna] < d[j][coluna])
        if cond:
            res.append(e[i]); i += 1
        else:
            res.append(d[j]); j += 1
    res.extend(e[i:]); res.extend(d[j:])
    return pd.DataFrame(res)


# ============================= Knapsack (problema da mochila) =============================

def knapsack(cursos, capacidade):
    n = len(cursos)
    dp = [[0] * (capacidade + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        impacto = cursos[i - 1]["impacto"]
        custo = cursos[i - 1]["custo"]
        for w in range(capacidade + 1):
            if custo <= w:
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - custo] + impacto)
            else:
                dp[i][w] = dp[i - 1][w]

    # reconstruir solução
    w = capacidade
    selecionados = []
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selecionados.append(cursos[i - 1])
            w -= cursos[i - 1]["custo"]

    selecionados.reverse()
    return selecionados


# ============================= Recomendação =============================

def recomendar_cursos(tempo_diario, dias=7, k=3):
    capacidade = tempo_diario * dias  # tempo total disponível em uma semana
    selecionados = knapsack(cursos, capacidade)

    if not selecionados:
        print("⚠️ Nenhum curso cabe no tempo disponível.\n")
        return pd.DataFrame([])

    # pegar os 3 com melhor impacto/custo dentro da solução ótima
    selecionados.sort(key=lambda x: x["impacto"] / x["custo"], reverse=True)
    top_k = selecionados[:k]

    df = pd.DataFrame(top_k)
    df["dias_para_concluir"] = (df["custo"] / max(tempo_diario, 1)).round(0).astype(int)
    df.loc[df["dias_para_concluir"] == 0, "dias_para_concluir"] = 1
    df["eficiencia"] = (df["impacto"] / df["custo"]).round(2)

    impacto_total = df["impacto"].sum()
    tempo_total = df["custo"].sum()

    print(f"\n📆 Horizonte: {dias} dias × {tempo_diario}h/dia = {capacidade}h disponíveis")
    print(f"💡 Impacto total: {impacto_total} | Tempo total usado: {tempo_total}h")
    print(f"⚙️ Eficiência média: {(impacto_total / tempo_total):.2f}\n")

    return df[["nome", "impacto", "custo", "dias_para_concluir", "eficiencia"]]


# ============================= Relatório =============================

def gerar_relatorio(usuario, cursos_recomendados):
    if cursos_recomendados.empty:
        print("Nenhum curso recomendado. Relatório não gerado.\n")
        return

    nome_arquivo = f"relatorio_{usuario['nome']}.txt"
    with open(nome_arquivo, "w", encoding="utf-8") as f:
        f.write(f"Relatório de Cursos Recomendados para {usuario['nome']}\n")
        f.write(f"Nível: {usuario['nivel']} | Tempo disponível: {usuario['tempo']}h/dia\n\n")
        f.write("Cursos sugeridos:\n")
        for _, row in cursos_recomendados.iterrows():
            f.write(f"- {row['nome']} (Impacto: {row['impacto']}, Custo: {row['custo']}h)\n")
    print(f"📄 Relatório gerado: {nome_arquivo}\n")


# ============================= Execução principal =============================

print("=== Sistema de Recomendação de Cursos ===")

while True:
    opcao = input("1 - Cadastrar | 2 - Login | 0 - Sair: \n")
    if opcao == "1":
        cadastrar_usuario()
    elif opcao == "2":
        user = login()
        if user:
            df_ordenado = merge_sort(df_cursos, "impacto", decrescente=True)
            print("\n📊 Cursos ordenados por impacto (decrescente):")
            print(df_ordenado)
            recomendados = recomendar_cursos(user["tempo"])
            print(recomendados)
            gerar_relatorio(user, recomendados)
    elif opcao == "0":
        print("Encerrando o sistema...")
        break
    else:
        print("Opção inválida.\n")


=== Sistema de Recomendação de Cursos ===
1 - Cadastrar | 2 - Login | 0 - Sair: 
2
=== Login ===

Usuário padrão:
  nome: admin
  senha: admin

Nome de usuário: admin
Senha: admin
✅ Login bem-sucedido! Bem-vindo(a), admin


📊 Cursos ordenados por impacto (decrescente):
                        nome  impacto  custo
0               Data Science       10      5
1           Machine Learning       10     30
2           APIs com FastAPI        9      4
3                 Kubernetes        9      5
4                     Python        9     15
5    Segurança da Informação        8     30
6              Java Avançado        8     20
7             Inglês Técnico        8      4
8                   Power BI        8     18
9          Liderança Técnica        7      3
10     Redes de Computadores        7     10
11             Docker Básico        7      3
12           Design Thinking        7      5
13                  IA Ética        7      4
14              Git e GitHub        6      5
15  Comuni

KeyboardInterrupt: Interrupted by user